In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

import torch
import torch.nn as nn
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
# from sklearn.model_selection import train_test_split, GridSearchCV
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_curve, auc, accuracy_score, confusion_matrix
# from sklearn.naive_bayes import MultinomialNB

import matplotlib.pyplot as plt
import seaborn as sns # plotting problem

!pip install contractions
import contractions # expanding contractions in text can't -> cannot

import os
import time
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.1 MB/s eta 0:00:00
/kaggle/input/amazon-product-review-spam-and-non-spam/Home_and_Kitchen/Home_and_Kitchen.json
/kaggle/input/amazon-product-review-spam-and-non-spam/part.json/part.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Electronics/Electronics.json
/kaggle/input/amazon-product-review-spam-and-non-spam/separate.json/separate.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Clothing_Shoes_and_Jewelry/Clothing_Shoes_and_Jewelry.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Sports_and_Outdoors/Sports_and_Outdoors.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json


In [2]:
torch.backends.cudnn.deterministic = True
random.seed(21)
np.random.seed(21)
torch.manual_seed(21)
torch.cuda.manual_seed_all(21)

In [3]:
import wandb
wandb.login(key = '02c8923278a3dc82932fafb9959cd6d7587dacc7')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
df = pd.read_json('/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json', lines=True).head(100000)
df = df.loc[:, ['reviewText', 'class']]
df

,reviewText,class
0,I love these felt nursery rhyme characters and...,1
1,I see no directions for its use. Therefore I h...,0
2,This is a great tool for any teacher using the...,1
3,"Great product, thank you! Our son loved the pu...",1
4,Although not as streamlined as the Algebra I m...,1
...,...,...
99995,Received this product in a timely fashion. I m...,0
99996,McFarlane Sports Series are fantastic and life...,1
99997,Fortune is a good figure. She has a very attra...,1
99998,I just thought that I'd jot a few words to let...,0


In [5]:
df['reviewText'] = df['reviewText'].apply(lambda x: contractions.fix(x))
df.loc[:, 'reviewText'] = df['reviewText'].str.lower()

# \W represents Special characters like "$" and "!!!"
df.loc[:, 'reviewText'] = df['reviewText'].str.replace('\W', ' ')

# \d represents Numeric digits like "19.99"
df.loc[:, 'reviewText'] = df['reviewText'].str.replace('\d', ' ')

df

,reviewText,class
0,i love these felt nursery rhyme characters and...,1
1,i see no directions for its use. therefore i h...,0
2,this is a great tool for any teacher using the...,1
3,"great product, thank you! our son loved the pu...",1
4,although not as streamlined as the algebra i m...,1
...,...,...
99995,received this product in a timely fashion. i m...,0
99996,mcfarlane sports series are fantastic and life...,1
99997,fortune is a good figure. she has a very attra...,1
99998,i just thought that i would jot a few words to...,0


In [6]:
y = df['class']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], y, test_size=0.33, random_state=21)

In [7]:
def pr_auc_score(y_test, y_pred):
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    pr_auc = auc(recall, precision)
    return pr_auc

In [8]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [9]:
class BERTCNN(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BERTCNN, self).__init__()
        self.bert = bert_model
        self.conv1 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=3)  # 768 is the hidden size of BERT-base
        self.conv2 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=4)
        self.conv3 = nn.Conv1d(in_channels=768, out_channels=100, kernel_size=5)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(300, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.last_hidden_state.permute(0, 2, 1)  # (N, L, D) -> (N, D, L), where D=768
        x1 = torch.relu(self.conv1(x)).max(dim=2)[0]  # (N, Co)
        x2 = torch.relu(self.conv2(x)).max(dim=2)[0] 
        x3 = torch.relu(self.conv3(x)).max(dim=2)[0] 
        x = torch.cat((x1, x2, x3), 1)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits

In [10]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [11]:
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve, auc

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Creating Datasets and DataLoaders
train_dataset = TextDataset(X_train, y_train.tolist(), tokenizer)
test_dataset = TextDataset(X_test, y_test.tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Parameters
NUM_CLASSES = 2
NUM_EPOCHS = 12
LEARNING_RATE = 0.001

wandb.init(project='RNN Spam Detection', name='CNN Spam Detection max')
# Model, Loss, Optimizer
model = BERTCNN(bert_model, NUM_CLASSES)
model = torch.nn.DataParallel(model)
model = model.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training the model
wandb.watch(model, criterion, log="all", log_freq=10)
start_time = time.time()
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}], Loss: {loss.item():.4f}')
    wandb.log({"epoch": epoch,"loss": loss})

# Evaluating the model
model.eval()
true_labels = []
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.tolist())
        predictions.extend(predicted.tolist())
        
elapsed_time = time.time() - start_time

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(true_labels, predictions))

precision, recall, _ = precision_recall_curve(true_labels, predictions)
pr_auc = auc(recall, precision)
print(f"PR AUC: {pr_auc:.4f}")

wandb.log({"test_accuracy": accuracy})
wandb.log({"test_pr_auc": pr_auc})

elapsed_time = time.time() - start_time
wandb.log({"run_time_sec": elapsed_time})
wandb.finish()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

wandb: Currently logged in as: mint21. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240520_065052-nbebuqps
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run CNN Spam Detection max
wandb: ⭐️ View project at https://wandb.ai/mint21/RNN%20Spam%20Detection
wandb: 🚀 View run at https://wandb.ai/mint21/RNN%20Spam%20Detection/runs/nbebuqps


Epoch [1], Loss: 0.2771
Epoch [2], Loss: 0.1959
Epoch [3], Loss: 0.1117
Epoch [4], Loss: 0.1478
Epoch [5], Loss: 0.1456
Epoch [6], Loss: 0.1017
Epoch [7], Loss: 0.1619
Epoch [8], Loss: 0.3595
Epoch [9], Loss: 0.1444
Epoch [10], Loss: 0.1285
Epoch [11], Loss: 0.1428
Epoch [12], Loss: 0.1617
Accuracy: 0.9333
              precision    recall  f1-score   support

           0       0.82      0.65      0.72      4420
           1       0.95      0.98      0.96     28580

    accuracy                           0.93     33000
   macro avg       0.88      0.81      0.84     33000
weighted avg       0.93      0.93      0.93     33000

PR AUC: 0.9720


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         epoch ▁▂▂▃▄▄▅▅▆▇▇█
wandb:          loss ▆▄▁▂▂▁▃█▂▂▂▃
wandb:  run_time_sec ▁
wandb: test_accuracy ▁
wandb:   test_pr_auc ▁
wandb: 
wandb: Run summary:
wandb:         epoch 11
wandb:          loss 0.16169
wandb:  run_time_sec 15099.25401
wandb: test_accuracy 0.93333
wandb:   test_pr_auc 0.97196
wandb: 
wandb: 🚀 View run CNN Spam Detection max at: https://wandb.ai/mint21/RNN%20Spam%20Detection/runs/nbebuqps
wandb: ⭐️ View project at: https://wandb.ai/mint21/RNN%20Spam%20Detection
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240520_065052-nbebuqps/logs
